In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [3]:
def resize_wafer_map(wm, target_size=(64, 64)):
    return tf.image.resize(np.expand_dims(wm, axis=-1), target_size, method='nearest').numpy()

In [4]:
def one_hot_img(wm):
    h, w = wm.shape[0], wm.shape[1]
    one_hot = np.zeros((h, w, 3))
    for i in range(h):
        for j in range(w):
            one_hot[i, j, int(wm[i, j])] = 1
    return one_hot

In [5]:
def add_noise(wm):
    h, w = wm.shape[0], wm.shape[1]
    n_change = int(0.01 * h * w)
    changed = 0
    noised = wm.copy()
    while changed < n_change:
        i, j = np.random.randint(0, h), np.random.randint(0, w)
        if wm[i, j, 0] == 0:  # If the point is on the wafer
            noised[i, j, 1] = int(not noised[i, j, 1])
            noised[i, j, 2] = int(not noised[i, j, 2])
            changed += 1
    return noised

In [16]:
def dataset(include_nonpattern=False, wm_dim=64):
    df = pd.read_pickle("data/LSWMD.pkl")
    df = df.drop(['trianTestLabel'], axis=1)
    df = df.drop(['waferIndex'], axis=1)

    df['failureNum'] = df.failureType
    mapping = {'Center': 0, 'Donut': 1, 'Edge-Loc': 2, 'Edge-Ring': 3,
               'Loc': 4, 'Near-full': 5, 'Random': 6, 'Scratch': 7, 'none': 8}
    df = df.replace({'failureNum': mapping})

    if include_nonpattern:
        failure_types = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring',
                         'Loc', 'Near-full', 'Random', 'Scratch', 'none']
        df = df[(df['failureNum'] >= 0) & (df['failureNum'] <= 8)]
    else:
        failure_types = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring',
                         'Loc', 'Near-full', 'Random', 'Scratch']
        df = df[(df['failureNum'] >= 0) & (df['failureNum'] <= 7)]

    df['waferMapResized'] = df.waferMap.apply(lambda w: resize_wafer_map(w, target_size=(wm_dim, wm_dim))).\
        apply(one_hot_img)

    for label in range(len(failure_types)):
        globals()['df_{}'.format(label)] = df[(df['failureNum'] == label)].sample(frac=1)

    test_ratio, valid_ratio = 0.2, 0.2
    for label in range(len(failure_types)):
        length = len(globals()['df_{}'.format(label)])
        idx_test = int(length * test_ratio)
        idx_valid = idx_test + int(length * valid_ratio)
        globals()['df_{}_test'.format(label)] = globals()['df_{}'.format(label)][:idx_test].reset_index()
        globals()['df_{}_valid'.format(label)] = globals()['df_{}'.format(label)][idx_test:idx_valid].reset_index()
        globals()['df_{}_train'.format(label)] = globals()['df_{}'.format(label)][idx_valid:].reset_index()
        del globals()['df_{}'.format(label)]

    max_class_len_train = np.max([len(globals()['df_{}_train'.format(i)]) for i in range(len(failure_types))])

    x_test, y_test = [], []
    for label in range(len(failure_types)):
        for wm in globals()['df_{}_test'.format(label)].waferMapResized:
            x_test.append(np.expand_dims(wm, axis=0))
            y_test.append(label)

    x_valid, y_valid = [], []
    for label in range(len(failure_types)):
        for wm in globals()['df_{}_valid'.format(label)].waferMapResized:
            x_valid.append(np.expand_dims(wm, axis=0))
            y_valid.append(label)

    x_train, y_train = [], []
    for label in range(len(failure_types)):
        length = len(globals()['df_{}_train'.format(label)])
        n_new = max_class_len_train - length
        for i in range(n_new):
            x = globals()['df_{}_train'.format(label)].waferMapResized.loc[np.random.choice(range(length))]
            if np.random.random() < 0.5:
                new_x = tf.image.flip_left_right(x).numpy()
            else:
                new_x = tf.image.rot90(x).numpy()
            new_x = add_noise(new_x)
            x_train.append(np.expand_dims(new_x, axis=0))
            y_train.append(label)
        for wm in globals()['df_{}_train'.format(label)].waferMapResized:
            x_train.append(np.expand_dims(wm, axis=0))
            y_train.append(label)

    x_test_arr = np.concatenate(x_test, axis=0)
    y_test_arr = np.array(y_test)
    x_valid_arr = np.concatenate(x_valid, axis=0)
    y_valid_arr = np.array(y_valid)
    x_train_arr = np.concatenate(x_train, axis=0)
    y_train_arr = np.array(y_train)
    del x_test, y_test, x_valid, y_valid, x_train, y_train

    y_test_arr = to_categorical(y_test_arr)
    y_valid_arr = to_categorical(y_valid_arr)
    y_train_arr = to_categorical(y_train_arr)

    return x_train_arr, y_train_arr, x_valid_arr, y_valid_arr, x_test_arr, y_test_arr

In [ ]:
x_train, y_train, x_valid, y_valid, x_test, y_test = dataset()

C:\Users\aeali\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\internals\managers.py:1911: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)
